In [80]:
# %pip install peft evaluate transformers Levenshtein ipywidgets
# %pip install protobuf==3.20.3
# !rm -rf /kaggle/working/cache

In [81]:
# X

import os
os.environ["TRANSFORMERS_DISABLE_CHAT_TEMPLATES"] = "1"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
os.environ["TRANSFORMERS_NO_ADDITIONAL_CHAT_TEMPLATES"] = "1"

In [82]:
from datasets import load_dataset, load_from_disk
# from UQA.canine_utils import preprocess_uqa, lora_config, print_trainable_parameters, normalize_answer, exact_match_score, f1_score, edit_distance_score, gold_answer, decode_prediction
from transformers import CanineTokenizer
from peft import LoraConfig, TaskType, get_peft_model
import re
import string
from collections import Counter
import numpy as np
import Levenshtein

from transformers import TrainingArguments, Trainer, TrainerCallback
import json
from huggingface_hub import HfApi, notebook_login, whoami

In [83]:
# notebook_login()
# whoami()

In [84]:
from transformers import CanineTokenizer, CanineForQuestionAnswering
import torch
model_name = 'google/canine-s'
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

tokenizer = CanineTokenizer.from_pretrained(model_name, use_fast=False, trust_remote_code=False)
model = CanineForQuestionAnswering.from_pretrained(model_name, trust_remote_code=False)

Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [85]:
uqa_dataset = load_dataset("uqa/UQA")
uqa_train = uqa_dataset["train"].shuffle(seed=42).select(range(80000))
uqa_val = uqa_dataset["validation"].shuffle(seed=42).select(range(2000))

In [86]:
len(uqa_dataset['train'])

124745

---

## Updated preprocessors!

Previously, we tried to apply the same approach we used in TYDIQA on UQA, the problem was the preprocessors were aligning the answer spans in units of **byte-level spans** instead of **character-level spans**. The calculations were adding byte-level offsets to the answer lengths, and since Urdu characters may be quantified in multiple bytes, the model was being fed the wrong spans -> GIGO!

In [87]:
MAX_SEQ_LENGTH = 384
DOC_STRIDE = 64

def preprocess_uqa(examples, tokenizer, max_length=MAX_SEQ_LENGTH, doc_stride=DOC_STRIDE, model_obj=None, indices=None):
    # Handle tokenizer/model limits safely
    tokenizer_max = getattr(tokenizer, "model_max_length", max_length)
    model_max = getattr(model_obj.config, "max_position_embeddings", None) if model_obj is not None else None
    max_allowed = max_length
    if tokenizer_max is not None and tokenizer_max > 0:
        max_allowed = min(max_allowed, tokenizer_max)
    if model_max is not None and model_max > 0:
        max_allowed = min(max_allowed, model_max)

    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]
    answers = examples["answer"]
    answer_starts = examples["answer_start"]

    encoded = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": [],
        "start_positions": [],
        "end_positions": [],
        "overflow_to_sample_mapping": []
    }

    for i, (question, context, answer, answer_start) in enumerate(zip(questions, contexts, answers, answer_starts)):
        example_idx = indices[i] if indices is not None else i

        # CANINE encodes to characters directly.
        # add_special_tokens=False gives us raw character IDs.
        question_ids = tokenizer.encode(question, add_special_tokens=False)
        context_ids = tokenizer.encode(context, add_special_tokens=False)

        # 1. Setup Targets (DIRECT MAPPING)
        # UQA answer_start is a CHARACTER index. CANINE tokens are CHARACTERS.
        # Therefore, answer_start maps 1:1 to the context_ids index.
        if answer and answer_start != -1:
            gold_char_start = answer_start
            gold_char_end = answer_start + len(answer) # Points to char AFTER the answer
        else:
            gold_char_start = -1
            gold_char_end = -1

        # 2. Calculate Window Size
        # [CLS] + Question + [SEP] + Context + [SEP]
        special_tokens_count = tokenizer.num_special_tokens_to_add(pair=True)
        max_context_length = max_allowed - len(question_ids) - special_tokens_count

        if max_context_length <= 0:
            # Edge case: Question is too long, skip or truncate question (skipping here for safety)
            continue

        # 3. Sliding Window Loop
        stride_step = max_context_length - doc_stride
        if stride_step <= 0: stride_step = max_context_length # Fallback if doc_stride is too big

        for chunk_start_idx in range(0, len(context_ids), stride_step):
            chunk_end_idx = min(chunk_start_idx + max_context_length, len(context_ids))
            context_chunk = context_ids[chunk_start_idx:chunk_end_idx]

            # Build inputs using tokenizer utility to handle special tokens correctly
            input_ids = tokenizer.build_inputs_with_special_tokens(question_ids, context_chunk)
            token_type_ids = tokenizer.create_token_type_ids_from_sequences(question_ids, context_chunk)
            attention_mask = [1] * len(input_ids)

            # Calculate Offset: Where does the context actually start in input_ids?
            # Typically: [CLS] (1) + Q_Len + [SEP] (1) = Start of Context
            # We calculate this dynamically to be safe:
            sep_indices = [k for k, x in enumerate(input_ids) if x == tokenizer.sep_token_id]
            if not sep_indices:
                continue # Should not happen
            context_offset_in_input = sep_indices[0] + 1

            # 4. Label Assignment
            # Check if the answer lies ENTIRELY within this specific chunk
            is_answer_in_chunk = (
                gold_char_start >= chunk_start_idx and
                gold_char_end <= chunk_end_idx and
                gold_char_start != -1
            )

            if is_answer_in_chunk:
                # Map global context index to local window index
                start_pos = context_offset_in_input + (gold_char_start - chunk_start_idx)
                # -1 because end_positions is usually inclusive in HF Trainers
                end_pos = context_offset_in_input + (gold_char_end - chunk_start_idx) - 1
            else:
                # Label as [CLS] (index 0) if answer is not here
                start_pos = 0
                end_pos = 0

            # 5. Padding
            pad_len = max_allowed - len(input_ids)
            if pad_len > 0:
                input_ids += [tokenizer.pad_token_id] * pad_len
                attention_mask += [0] * pad_len
                token_type_ids += [0] * pad_len

            # 6. Final Safety Truncation (just in case)
            if len(input_ids) > max_allowed:
                input_ids = input_ids[:max_allowed]
                attention_mask = attention_mask[:max_allowed]
                token_type_ids = token_type_ids[:max_allowed]
                # If labels were pushed out by truncation, reset to 0
                if start_pos >= max_allowed or end_pos >= max_allowed:
                    start_pos = 0
                    end_pos = 0

            encoded["input_ids"].append(input_ids)
            encoded["attention_mask"].append(attention_mask)
            encoded["token_type_ids"].append(token_type_ids)
            encoded["start_positions"].append(start_pos)
            encoded["end_positions"].append(end_pos)
            encoded["overflow_to_sample_mapping"].append(example_idx)

            # Break loop if this chunk reached the end of the context
            if chunk_end_idx >= len(context_ids):
                break

    return encoded


In [88]:
# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,
    r=16,   # changed from 8
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["query", "value", "key"],
    bias="none",
    modules_to_save=["qa_outputs"],
)

def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")


In [89]:
# preprocess the train and val splits
processed_train = uqa_train.map(lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), batched=True, remove_columns=uqa_train.column_names, with_indices=True)
processed_val = uqa_val.map(lambda examples, indices: preprocess_uqa(examples, tokenizer, indices=indices), batched=True, remove_columns=uqa_val.column_names, with_indices=True)

In [90]:
# processed_train

In [91]:
# processed_val

In [92]:
processed_train.save_to_disk("/kaggle/working/cache/processed_train_uqa")
processed_val.save_to_disk("/kaggle/working/cache/processed_val_uqa")   # cached it


processed_train = load_from_disk("/kaggle/working/cache/processed_train_uqa")
processed_val = load_from_disk("/kaggle/working/cache/processed_val_uqa")

Saving the dataset (0/2 shards):   0%|          | 0/233583 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6317 [00:00<?, ? examples/s]

In [93]:
import random
import pandas as pd

# Set display options to ensure Urdu text is readable
pd.set_option('display.max_colwidth', None)

def verify_training_data(dataset, original_dataset, tokenizer, num_samples=1000):
    print(f"🔍 Auditing {num_samples} random training samples...")
    
    # Filter for samples that actually HAVE an answer marked (start_positions > 0)
    # We want to check if the non-zero labels are correct.
    indices_with_answers = [i for i, x in enumerate(dataset["start_positions"]) if x > 0]
    
    if not indices_with_answers:
        print("⚠️ No samples with answers found! (Is the dataset all [CLS] labels?)")
        return

    sample_indices = random.sample(indices_with_answers, min(num_samples, len(indices_with_answers)))
    
    results = []
    
    for idx in sample_indices:
        # 1. Get the Processed Data (What the model sees)
        proc = dataset[idx]
        input_ids = proc["input_ids"]
        start = proc["start_positions"]
        end = proc["end_positions"]
        
        # 2. Get the Original Data (The Truth)
        orig_idx = proc["overflow_to_sample_mapping"]
        orig = original_dataset[orig_idx]
        gold_answer = orig["answer"]
        question = orig["question"]
        
        # 3. Decode the Label Span
        # We perform the exact slice the model is being taught to predict
        label_ids = input_ids[start : end + 1]
        decoded_label = tokenizer.decode(label_ids, skip_special_tokens=True)
        
        # 4. Compare
        # Strip whitespace for fair comparison
        is_match = (decoded_label.strip() == gold_answer.strip())
        
        results.append({
            "Original Index": orig_idx,
            "Question": question,
            "Gold Answer": gold_answer,
            "Label Decoded (Model Target)": decoded_label,
            "Status": "Match" if is_match else "❌ FAIL"
        })

    # Display Table
    df = pd.DataFrame(results)
    display(df)
    
    # Summary
    pass_count = len(df[df["Status"] == "Match"])
    print(f"Pass: {pass_count} / {num_samples}")
    

# Run the verification
verify_training_data(processed_train, uqa_train, tokenizer)

🔍 Auditing 1000 random training samples...


,Original Index,Question,Gold Answer,Label Decoded (Model Target),Status
0,74484,مدرسہ لفظ کی ابتداء کیا ہے؟,عربی,عربی,Match
1,33643,""" سیاہ فام"" لوگوں کی اصطلاح میں کون شامل ہیں؟",افریقی ، رنگین اور ایشیائی,افریقی ، رنگین اور ایشیائی,Match
2,2320,اٹلانٹک سٹی کے ساحل پر واقع لائٹ ہاؤس کا نام کیا ہے؟,ایبسیکن لائٹ ہاؤس,ایبسیکن لائٹ ہاؤس,Match
3,39786,شوپن وارسا میں کب واپس آیا؟,ستمبر 1829,ستمبر 1829,Match
4,77396,اولمپک کھیل کب شروع ہوئے؟,776 قبل مسیح,776 قبل مسیح,Match
...,...,...,...,...,...
995,16843,کون سی قسم کی ورزش دماغ کے لیے سب سے زیادہ فائدہ مند ثابت ہوئی ہے؟,ایروبک مشقیں,ایروبک مشقیں,Match
996,56850,مائکروسکوپی کب انتہائی مخصوص ہوتی ہے؟,جب اینٹی باڈی پر مبنی تکنیک کے ساتھ مل کر استعمال کیا جاتا ہے۔,جب اینٹی باڈی پر مبنی تکنیک کے ساتھ مل کر استعمال کیا جاتا ہے۔,Match
997,55479,ڈی سی کا مخفف کیا ہے؟,ڈائریکٹ کرنٹ,ڈائریکٹ کرنٹ,Match
998,44542,لیمب کا آئینہ کے بارے میں کیا نظریہ تھا؟,خطرناک,خطرناک,Match


Pass: 1000 / 1000


### Checking how many rows have unanswerable questions...

In [94]:
if 'processed_train' not in globals():
    try:
        print("Loading processed training set from disk...")

        processed_train = load_from_disk("/kaggle/working/cache/processed_train_uqa")
    except Exception as e:
        print(f"Error loading data: {e}")
        print("Please ensure 'processed_val' is defined or the path is correct.")


print(f"Analyzing {len(processed_train)} training samples...")

unanswerable_count = 0
total_count = len(processed_train)

for i in range(total_count):

    if processed_train[i]["start_positions"] == 0 and processed_train[i]["end_positions"] == 0:
        unanswerable_count += 1

percentage = (unanswerable_count / total_count) * 100


print("-" * 40)
print(f"Total Samples:       {total_count}")
print(f"Unanswerable ([CLS]): {unanswerable_count}")
print(f"Baseline Percentage:  {percentage:.2f}%")
print("-" * 40)

Analyzing 233583 training samples...
----------------------------------------
Total Samples:       233583
Unanswerable ([CLS]): 174798
Baseline Percentage:  74.83%
----------------------------------------


In [95]:
if 'processed_val' not in globals():
    try:
        print("Loading processed validation set from disk...")

        processed_val = load_from_disk("/kaggle/working/cache/processed_val_uqa")
    except Exception as e:
        print(f"Error loading data: {e}")
        print("Please ensure 'processed_val' is defined or the path is correct.")


print(f"Analyzing {len(processed_val)} validation samples...")

unanswerable_count = 0
total_count = len(processed_val)

for i in range(total_count):

    if processed_val[i]["start_positions"] == 0 and processed_val[i]["end_positions"] == 0:
        unanswerable_count += 1

percentage = (unanswerable_count / total_count) * 100

print("-" * 40)
print(f"Total Samples:       {total_count}")
print(f"Unanswerable ([CLS]): {unanswerable_count}")
print(f"Baseline Percentage:  {percentage:.2f}%")
print("-" * 40)


Analyzing 6317 validation samples...
----------------------------------------
Total Samples:       6317
Unanswerable ([CLS]): 4834
Baseline Percentage:  76.52%
----------------------------------------


There is a **severe** class imbalance issue here. Aorund 75% of the data is unanswerable!

Need to downsample the training dataset...

In [96]:
from datasets import concatenate_datasets

def create_balanced_dataset(dataset):
    print(f"Original Training Size: {len(dataset)}")
    
    # 1. Split into Positive (Has Answer) and Negative (No Answer)
    # In SQuAD preprocessing, start_positions=0 indicates [CLS] / No Answer
    positives = dataset.filter(lambda x: x["start_positions"] > 0)
    negatives = dataset.filter(lambda x: x["start_positions"] == 0)
    
    n_pos = len(positives)
    n_neg = len(negatives)
    
    print(f"  - Answerable Samples:   {n_pos}")
    print(f"  - Unanswerable Samples: {n_neg}")
    
    # 2. Downsample Negatives to match Positives (1:1 Ratio)
    # We shuffle first to get a random distribution
    negatives = negatives.shuffle(seed=42)
    
    # Select only as many negatives as we have positives
    negatives_balanced = negatives.select(range(n_pos))
    
    print(f"  - Negatives Kept:       {len(negatives_balanced)}")
    
    # 3. Combine and Shuffle
    balanced_dataset = concatenate_datasets([positives, negatives_balanced])
    balanced_dataset = balanced_dataset.shuffle(seed=42)
    
    print(f"Balanced Training Size: {len(balanced_dataset)}")
    
    return balanced_dataset


In [97]:
balanced_train = create_balanced_dataset(processed_train)

Original Training Size: 233583
  - Answerable Samples:   58785
  - Unanswerable Samples: 174798
  - Negatives Kept:       58785
Balanced Training Size: 117570


---

In [98]:
balanced_train.save_to_disk("/kaggle/working/cache/balanced_train_uqa")   # cached it


balanced_train = load_from_disk("/kaggle/working/cache/balanced_train_uqa")

Saving the dataset (0/1 shards):   0%|          | 0/117570 [00:00<?, ? examples/s]

In [99]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")



In [100]:
# build LoRA model

peft_model = get_peft_model(model, lora_config)
peft_model.gradient_checkpointing_enable()
print_trainable_parameters(peft_model)

trainable params: 1033730 || all params: 133118212 || trainable%: 0.7765503941714602


In [101]:
def normalize_answer(text):
    text = (text or "").lower()
    def remove_articles(s):
        return re.sub(r"\b(a|an|the)\b", " ", s)
    def remove_punctuation(s):
        return "".join(ch for ch in s if ch not in string.punctuation)
    def white_space_fix(s):
        return " ".join(s.split())
    return white_space_fix(remove_articles(remove_punctuation(text)))

def exact_match_score(prediction, ground_truth):
    return float(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    pred_tokens = normalize_answer(prediction).split()
    gold_tokens = normalize_answer(ground_truth).split()
    if not gold_tokens:
        return 1.0 if not pred_tokens else 0.0
    if not pred_tokens:
        return 0.0
    common = Counter(pred_tokens) & Counter(gold_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gold_tokens)
    return 2 * precision * recall / (precision + recall)

def decode_prediction(input_ids, start_idx, end_idx, tokenizer):
    cls_index = input_ids.index(tokenizer.cls_token_id)
    if start_idx == cls_index and end_idx == cls_index:
        return "" # Empty answer
    if start_idx > end_idx:
        return "" # Invalid range -> Empty answer
    
    # Ensure indices are within bounds
    start_idx = max(start_idx, 0)
    end_idx = min(end_idx, len(input_ids) - 1)
    
    text = tokenizer.decode(input_ids[start_idx:end_idx + 1], skip_special_tokens=True)
    return text.strip()

def gold_answer(example):
    if example["answer_start"] == -1:
        return ""
    return example["answer"]

def edit_distance_score(prediction, ground_truth):
    return Levenshtein.ratio(normalize_answer(prediction), normalize_answer(ground_truth))


def evaluate_checkpoint(checkpoint_path=None, model_instance=None, eval_dataset=None):
    """Evaluate either a checkpoint path (loads model) or a provided model instance.

    - checkpoint_path: path to checkpoint folder
    - model_instance: an in-memory model (preferably a PeftModel or CanineForQuestionAnswering)
    - eval_dataset: optional dataset to evaluate; if None the default processed_val will be used
    """
    if eval_dataset is None:
        eval_dataset = processed_val

    # If a model_instance is given, use it directly (avoid re-loading a fresh base model)
    if model_instance is not None:
        eval_model = model_instance
    else:
        base_model = CanineForQuestionAnswering.from_pretrained(model_name, trust_remote_code=False)
        eval_model = get_peft_model(base_model, lora_config)
        # Try loading adapter weights; fall back to PeftModel.from_pretrained if needed
        try:
            eval_model.load_adapter(checkpoint_path)
        except Exception:
            from peft import PeftModel
            eval_model = PeftModel.from_pretrained(base_model, checkpoint_path)

    eval_model.to(device)

    eval_args = TrainingArguments(
        # Small evaluation config; uses cpu/mps if no gpu during eval
        output_dir="outputs/canine-s-uqa",
        per_device_eval_batch_size=16,
        dataloader_drop_last=False,
        fp16=True,
        bf16=False,
        report_to="none",
    )

    # Run evaluation via a lightweight Trainer so prediction loop is standard
    eval_trainer = Trainer(
        model=eval_model,
        args=eval_args,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    predictions = eval_trainer.predict(eval_dataset)
    start_logits, end_logits = predictions.predictions
    best_predictions = {}
    for feature_index, feature in enumerate(eval_dataset):
        sample_idx = int(feature["overflow_to_sample_mapping"])
        input_ids = feature["input_ids"]
        start_idx = int(np.argmax(start_logits[feature_index]))
        end_idx = int(np.argmax(end_logits[feature_index]))
        score = float(start_logits[feature_index][start_idx] + end_logits[feature_index][end_idx])
        prediction_text = decode_prediction(input_ids, start_idx, end_idx, tokenizer=tokenizer)
        stored = best_predictions.get(sample_idx)
        if stored is None or score > stored[0]:
            best_predictions[sample_idx] = (score, prediction_text)

    em_scores = []
    f1_scores = []
    edit_dist_scores = []
    for sample_idx, (_, prediction_text) in best_predictions.items():
        reference = gold_answer(uqa_val[int(sample_idx)])
        em_scores.append(exact_match_score(prediction_text, reference))
        f1_scores.append(f1_score(prediction_text, reference))
        edit_dist_scores.append(edit_distance_score(prediction_text, reference))

    em = float(np.mean(em_scores)) if em_scores else 0.0
    f1 = float(np.mean(f1_scores)) if f1_scores else 0.0
    edit_dist = float(np.mean(edit_dist_scores)) if edit_dist_scores else 0.0

    # ---
    print("\n👀 VISUAL AUDIT (Top 5 Predictions):")
    print(f"{'Question':<50} | {'Gold':<20} | {'Prediction':<20}")
    print("-" * 100)
    
    # Print first 5 items from best_predictions to the console log
    count = 0
    for sample_idx, (score, pred_text) in best_predictions.items():
        if count >= 5: break
        # Get question text (you might need to store it or fetch from dataset)
        raw_gold = gold_answer(uqa_val[int(sample_idx)])
        print(f"Sample #{sample_idx:<8} | {raw_gold:<20} | {pred_text:<20}")
        count += 1
    print("-" * 100 + "\n")
    # ---
    
    print(f"Examples evaluated: {len(em_scores)}")
    print(f"Exact Match: {em * 100:.2f}")
    print(f"F1: {f1 * 100:.2f}")
    print(f"Edit Distance (normalized): {edit_dist * 100:.2f}")
    return {"exact_match": em, "f1": f1, "edit_distance": edit_dist}


In [102]:
training_args = TrainingArguments(
    output_dir="outputs/canine-s-uqa",

    

    per_device_train_batch_size=4,
    per_device_eval_batch_size=16,

    gradient_accumulation_steps=4,
    gradient_checkpointing=True,

    num_train_epochs=2,
    learning_rate=3e-4,     # trying a higher lr
    weight_decay=0.00,      # trying 0 weight decay
    lr_scheduler_type="constant",   # CHANGED (Don't let LR decay to 0)
    warmup_ratio=0.05,              # Small warmup for stability
    
    eval_strategy="no",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    logging_steps=50,
    fp16=True,
    bf16=False,
    report_to="none",
    push_to_hub=True,
    hub_model_id="VohraAK/canine-s-uqa",
    hub_strategy="checkpoint",
    )

class CustomEvalCallback(TrainerCallback):
    def __init__(self, eval_func, eval_dataset, use_in_memory_model=True, verbose=True):
        self.eval_func = eval_func
        self.eval_dataset = eval_dataset
        self.use_in_memory_model = use_in_memory_model
        self.verbose = verbose
        # trainer reference (set after trainer exists)
        self.trainer = None

    def on_save(self, args, state, control, model=None, **kwargs):
        checkpoint_path = f"{args.output_dir}/checkpoint-{state.global_step}"
        if self.verbose:
            print(f"\n🔍 Running custom evaluation at step {state.global_step}...")

        # Prefer evaluating the in-memory trainer model (fast + avoids re-loading)
        if self.use_in_memory_model and self.trainer is not None:
            if self.verbose:
                print("Using in-memory model for evaluation (no reloading).")
            try:
                metrics = self.eval_func(checkpoint_path=None, model_instance=self.trainer.model, eval_dataset=self.eval_dataset)
            except Exception as e:
                print("⚠️ in-memory evaluation failed, falling back to checkpoint load:", e)
                metrics = self.eval_func(checkpoint_path)
        else:
            metrics = self.eval_func(checkpoint_path)

        # record metrics in state.log_history
        state.log_history.append({
            "step": state.global_step,
            "eval_exact_match": metrics.get("exact_match"),
            "eval_f1": metrics.get("f1"),
            "eval_edit_distance": metrics.get("edit_distance"),
        })

        if self.verbose:
            print(f"✅ Step {state.global_step}: EM={metrics.get('exact_match',0)*100:.2f}, F1={metrics.get('f1',0)*100:.2f}, EditDist={metrics.get('edit_distance',0)*100:.2f}")

        # Update trainer_state.json to include custom metrics
        state_path = f"{checkpoint_path}/trainer_state.json"
        try:
            with open(state_path, 'r') as f:
                state_dict = json.load(f)
            state_dict['log_history'] = state.log_history
            with open(state_path, 'w') as f:
                json.dump(state_dict, f, indent=2)
            if self.verbose:
                print(f"💾 Updated trainer_state.json with custom metrics")
        except Exception as e:
            if self.verbose:
                print(f"⚠️  Warning: Could not update trainer_state.json: {e}")

        try:
            if self.verbose:
                print(f"☁️  Pushing checkpoint-{state.global_step} to Hub...")
            api = HfApi()
            api.upload_folder(
                folder_path=checkpoint_path,
                repo_id=args.hub_model_id,
                path_in_repo=f"checkpoint-{state.global_step}",
                commit_message=f"Add checkpoint {state.global_step} (EM={metrics.get('exact_match',0)*100:.1f}%, F1={metrics.get('f1',0)*100:.1f}%)",
                repo_type="model"
            )
            if self.verbose:
                print(f"✅ Pushed checkpoint-{state.global_step} to Hub")
        except Exception as e:
            if self.verbose:
                print(f"⚠️  Warning: Could not push to Hub: {e}")

        return control

In [103]:
trainer_cb = CustomEvalCallback(evaluate_checkpoint, processed_val, use_in_memory_model=True)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=balanced_train,
    eval_dataset=processed_val,
    callbacks=[trainer_cb],
)


No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [104]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
50,5.808600
100,5.547500
150,5.377200
200,5.228000
250,5.089500
300,4.927800
350,4.830500
400,4.735100
450,4.766400
500,4.620400



🔍 Running custom evaluation at step 500...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipykernel_111/2115667162.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used inste


👀 VISUAL AUDIT (Top 5 Predictions):
Question                                           | Gold                 | Prediction          
----------------------------------------------------------------------------------------------------
Sample #0        | مغربی کیپ صوبہ       | پلیسس               
Sample #1        |                      |                     
Sample #2        |                      |                     
Sample #3        | ہیوگس مفروضہ         | گالیکنز اور         
Sample #4        |                      |                     
----------------------------------------------------------------------------------------------------

Examples evaluated: 2000
Exact Match: 28.65
F1: 29.14
Edit Distance (normalized): 30.37
✅ Step 500: EM=28.65, F1=29.14, EditDist=30.37
💾 Updated trainer_state.json with custom metrics
☁️  Pushing checkpoint-500 to Hub...
✅ Pushed checkpoint-500 to Hub


...


🔍 Running custom evaluation at step 10500...


Some weights of CanineForQuestionAnswering were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipykernel_111/2115667162.py:88: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  eval_trainer = Trainer(
No label_names provided for model class `PeftModelForQuestionAnswering`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used inste


👀 VISUAL AUDIT (Top 5 Predictions):
Question                                           | Gold                 | Prediction          
----------------------------------------------------------------------------------------------------
Sample #0        | مغربی کیپ صوبہ       | جنوبی افریقہ میں سابقہ ہوگنوٹ فارم کہاں پائے جا سکتے ہیں؟، ڈو پلیسس
Sample #1        |                      |                     
Sample #2        |                      |                     
Sample #3        | ہیوگس مفروضہ         |                     
Sample #4        |                      |                     
----------------------------------------------------------------------------------------------------

Examples evaluated: 2000
Exact Match: 32.50
F1: 32.64
Edit Distance (normalized): 32.80
✅ Step 10500: EM=32.50, F1=32.64, EditDist=32.80
💾 Updated trainer_state.json with custom metrics
☁️  Pushing checkpoint-10500 to Hub...
✅ Pushed checkpoint-10500 to Hub


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

---

### Diagnosing Preprocessing Functions!!!

These functions are just analysing the preprocessing logic above, they're just using the base model, NOT our trained model...

In [ ]:
# Diagnostic cell (fixed): Investigate preprocessing and truncation for many samples
import random
import pandas as pd
from transformers import AutoTokenizer

# Set display options to see full Urdu text
pd.set_option('display.max_colwidth', None)

try:
    tokenizer = AutoTokenizer.from_pretrained("google/canine-s")
except Exception:
    tokenizer = None

num_samples = 20000  # Number of samples to check
results = []

for split_name, orig_data, proc_data in [
    ("train", uqa_train, processed_train),
    ("val", uqa_val, processed_val)
]:
    # Sample random indices
    if len(proc_data) < num_samples:
        current_indices = range(len(proc_data))
    else:
        current_indices = random.sample(range(len(proc_data)), num_samples)

    for idx in current_indices:
        proc = proc_data[idx]
        # Use overflow_to_sample_mapping to get the correct original index
        orig_idx = proc["overflow_to_sample_mapping"]
        orig = orig_data[orig_idx]

        input_ids = proc["input_ids"]
        start_pos = proc["start_positions"]
        end_pos = proc["end_positions"]

        gold_answer = orig.get("gold_answer", orig.get("answer", ""))
        question = orig.get("question", "")

        # Decode input_ids to text (for debugging context)
        if tokenizer:
            decoded_text = tokenizer.decode(input_ids, skip_special_tokens=False)
        else:
            decoded_text = str(input_ids)

        # Extract predicted answer span
        if 0 <= start_pos < len(input_ids) and 0 <= end_pos < len(input_ids):
            if tokenizer:
                pred_span = tokenizer.decode(input_ids[start_pos:end_pos+1], skip_special_tokens=True)
            else:
                pred_span = str(input_ids[start_pos:end_pos+1])
        else:
            pred_span = "[CLS]" # Represents no answer found in this chunk or invalid

        # Check if pred_span matches gold answer
        # We strip() to ignore minor whitespace differences
        pred_matches_gold = pred_span.strip() == gold_answer.strip()

        # Check if gold is even reachable in this chunk
        gold_in_decoded = gold_answer in decoded_text

        results.append({
            "Split": split_name,
            "Question": question,
            "Gold Answer": gold_answer,
            "Extracted Answer": pred_span,
            "Match": pred_matches_gold,
            "Gold Reachable": gold_in_decoded,
            "orig_idx": orig_idx
        })

# Create DataFrame
results_df = pd.DataFrame(results)

# --- SIDE BY SIDE COMPARISON ---

# 1. Filter for Solvable Mismatches (Gold was there, but we predicted wrong)
problem_cases = results_df[
    (results_df["Gold Reachable"] == True) &
    (results_df["Match"] == False)
][["Question", "Gold Answer", "Extracted Answer", "Split"]]

print(f"🔍 Checked {len(results_df)} samples.")
print(f"❌ Found {len(problem_cases)} cases where Gold was present but Extraction failed.")

print("\n📊 Side-by-Side Comparison (Top 20 Failures):")
display(problem_cases.head(50))

print("\n✅ Side-by-Side Comparison (First 10 Rows - Mixed):")
display(results_df[["Question", "Gold Answer", "Extracted Answer", "Match"]].head(50))